<a href="https://colab.research.google.com/github/50-Course/swarm-optimizers/blob/main/SwarmOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*italicized text*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Todo: use numpy.random.seed somewhere
# You must ensure that all the results are pre-generated and repeatable2,
# otherwise you may lose marks.
dir(np.random.seed)


['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

SyntaxError: ignored

In [9]:
!git config --global user.email erioluwaadeodu@my.uopeople.edu
!git config --global user.name 'Eri Adeodu'
!git branch -M main


In [ ]:
# Validation code
x = np.array([4.19, 11.57, 6.69, 10.65])
print("Objective function output, f(x) = ", f(x))
print("Objective function output, g1(x) = ", g1(x))
print("Objective function output, g2(x) = ", g2(x))
print("Objective function output, g3(x) = ", g3(x))
print("Objective function output, g4(x) = ", g4(x))
print("Objective function output, g5(x) = ", g5(x))
print("Objective function output, g6(x) = ", g6(x))
print("Objective function output, g7(x) = ", g7(x))


NameError: ignored